In [1]:
url = "http://data.neo4j.com/intro-neo4j/actors.csv"

In [3]:
!pip install py2neo

  Using cached https://files.pythonhosted.org/packages/fc/d0/7fc3a811e011d4b388be48a0e381db8d990042df54aa4ef4599a31d39853/ipaddress-1.0.22-py2.py3-none-any.whl


In [4]:
from py2neo import Graph, Node, Relationship
import pandas as pd

In [5]:
neo_server = "52.15.124.208"
user = "neo4j"
passw = "digityser"

In [7]:
graph = Graph(host=neo_server, auth=(user, passw))

In [8]:
# clear your database or delete the database file, stop and start neo4j service
graph.delete_all()

In [ ]:
# introducing spaces

In [177]:
%%writefile ../neo4j/import/sample1.csv
id,name,birthYear,carBrand
0,  Kevin Albert ,1981,Renault
1,Susan Sarandon,1946,Renault
2,Julia Roberts,1967,VW
3,Holly Hunter,1958,BMW
4,Ariel Winter,1998,BMW
5,Goldie Hawn,1945,Tesla
6,Diane Keaton,1946,Tesla

Overwriting ../neo4j/import/sample1.csv


In [166]:
query = """
LOAD CSV WITH HEADERS FROM 'file:///sample1.csv' AS line
WITH line
LIMIT 1
RETURN line
"""

In [167]:
data = graph.run(query)

In [168]:
next(data)

<Record line={'carBrand': 'Renault', 'birthYear': '1981', 'name': '  Kevin Albert  ', 'id': '0'}>

## use py2neo for loading csv into neo4j

In [178]:
query = """
USING PERIODIC COMMIT 2
LOAD CSV WITH HEADERS FROM 'file:///sample1.csv' AS line FIELDTERMINATOR ','
MERGE (p: Person {id: toInteger(trim(line.id)),
                  name: trim(line.name),
                  birthyear: toInteger(line.birthYear)})
"""
graph.run(query)

In [179]:
query = """
USING PERIODIC COMMIT 2
LOAD CSV WITH HEADERS FROM 'file:///sample1.csv' AS line FIELDTERMINATOR ','
MERGE (c: Car {brand: line.carBrand})
"""
graph.run(query)

In [180]:
query = """
USING PERIODIC COMMIT 2
LOAD CSV WITH HEADERS FROM 'file:///sample1.csv' AS line FIELDTERMINATOR ','
MATCH (p: Person {name: trim(line.name)})
MATCH (c: Car {brand: line.carBrand})
MERGE (p)-[:DRIVES]->(c)
"""
graph.run(query)

## reading data to a dataframe is better to use %cypher

In [9]:
!pip install ipython-cypher pandas

In [ ]:
neo_server = "52.15.124.208"
user = "neo4j"
passw = "digityser"

In [157]:
%config CypherMagic.uri="http://"+user+":"+passw+"@"+neo_server+":7474/db/data"

In [82]:
%load_ext cypher

In [181]:
data = %cypher \
MATCH (p:Person)-[:DRIVES]->(c:Car) \
RETURN p.name, p.birthyear, c.brand

7 rows affected.


In [89]:
import pandas as pd

In [182]:
df = data.get_dataframe()
df

,p.name,p.birthyear,c.brand
0,Susan Sarandon,1946,Renault
1,Kevin Albert,1981,Renault
2,Diane Keaton,1946,Tesla
3,Goldie Hawn,1945,Tesla
4,Julia Roberts,1967,VW
5,Ariel Winter,1998,BMW
6,Holly Hunter,1958,BMW


In [183]:
df.loc[1][0]

'Kevin Albert'

# let's try loading 2016 data from DataforbetterHealth

In [1]:
from py2neo import Graph

In [2]:
neo_server = "52.15.124.208"
user = "neo4j"
passw = "digityser"
file = "df_clean_v22016.csv"

In [3]:
graph = Graph(host=neo_server, auth=(user, passw))

In [22]:
graph.delete_all()

In [5]:
# uploaded a csv 283MB /import "df_clean_v22016.csv"
!head -n 1 ../neo4j/import/df_clean_v22016.csv

Province,hosp_serv_id,reimbt_cat_id,drug_code,realization_date,amount_reimb,amount_not_reimb,type_drug_code,famille_produit_pharma,drug_name_aggregated,conditionnement,mode_administration,date_debut_rembourse,statut_produit_pharma,code_atc,code_atc_5,code_atc_4,code_atc_3,code_atc_1,DDD,nombre_prises,orphan_flag,chapter_IV_bis_flag,link_same_tablet,dbegin,dend,reimbt_cat_desc_nl,reimbt_crit_long,reimbt_crit_short,reimbt_cat_fixed_rate_yn,fee_cat,quantity_returned,quantity_delivered


In [6]:
query = """
LOAD CSV WITH HEADERS FROM 'file:///"""+file+"""' AS line
WITH line
LIMIT 1
RETURN line
"""

In [7]:
data = graph.run(query)

In [8]:
next(data)

<Record line={'mode_administration': 'INJECTION', 'quantity_returned': '0', 'DDD': '1.0', 'chapter_IV_bis_flag': 'False', 'drug_code': '0716191', 'fee_cat': 'A', 'famille_produit_pharma': 'Spécialité pharmaceutique', 'reimbt_cat_id': '750724', 'conditionnement': '* pr. amp. I.V. 1 x 1 mg/2 ml', 'realization_date': '2016', 'dend': '2099-12-31', 'reimbt_crit_long': 'A', 'hosp_serv_id': '2', 'drug_name_aggregated': 'RAPIFEN', 'statut_produit_pharma': 'orig', 'orphan_flag': 'False', 'quantity_delivered': '1', 'reimbt_cat_fixed_rate_yn': 'False', 'link_same_tablet': '-15373.0', 'Province': 'Antwerpen', 'type_drug_code': 'Unit', 'amount_reimb': '0.45', 'code_atc_5': 'N01AH', 'code_atc_4': 'N01A', 'code_atc_3': 'N01', 'code_atc': 'N01AH02', 'nombre_prises': '0.0', 'code_atc_1': 'N', 'reimbt_crit_short': 'A', 'date_debut_rembourse': '1984-03-01', 'dbegin': '1985-04-01', 'amount_not_reimb': '0.0', 'reimbt_cat_desc_nl': 'Farmaceutische specialiteit van categorie A'}>

## loading data

In [23]:
query = """
CREATE CONSTRAINT ON (h: Hospital) ASSERT (h.hosp_serv_id) IS UNIQUE
"""
graph.run(query)

In [24]:
query = """
USING PERIODIC COMMIT 100
LOAD CSV WITH HEADERS FROM 'file:///"""+file+"""' AS line FIELDTERMINATOR ','
MERGE (h: Hospital {hosp_serv_id: toInteger(line.hosp_serv_id)})
"""
graph.run(query)

In [25]:
query = """
CREATE CONSTRAINT ON (p: Province) ASSERT (p.name) IS UNIQUE
"""
graph.run(query)

In [26]:
query = """
USING PERIODIC COMMIT 100
LOAD CSV WITH HEADERS FROM 'file:///"""+file+"""' AS line FIELDTERMINATOR ','
MERGE (p: Province {name: line.Province})
"""
graph.run(query)

In [27]:
query = """
CREATE CONSTRAINT ON (d: Drug) ASSERT (d.code) IS UNIQUE
"""
graph.run(query)

In [28]:
# query = """
# USING PERIODIC COMMIT 100
# LOAD CSV WITH HEADERS FROM 'file:///"""+file+"""' AS line FIELDTERMINATOR ','
# MERGE (d: Drug {code: line.drug_code})
# """
# graph.run(query)

In [29]:
query = """
USING PERIODIC COMMIT 100
LOAD CSV WITH HEADERS FROM 'file:///"""+file+"""' AS line FIELDTERMINATOR ','
MERGE (d: Drug {code: line.drug_code,
                type_drug_code: line.type_drug_code,
                drug_name_aggregated: line.drug_name_aggregated,
                code_atc: line.code_atc})
"""
graph.run(query)

## creating relations

In [30]:
query = """
USING PERIODIC COMMIT 100
LOAD CSV WITH HEADERS FROM 'file:///"""+file+"""' AS line FIELDTERMINATOR ','
MATCH (h: Hospital {hosp_serv_id: toInteger(line.hosp_serv_id)})
MATCH (p: Province {name: line.Province})
MERGE (h)-[:LOCATED]->(p)
"""
graph.run(query)

In [31]:
query = """
USING PERIODIC COMMIT 100
LOAD CSV WITH HEADERS FROM 'file:///"""+file+"""' AS line FIELDTERMINATOR ','
MATCH (h: Hospital {hosp_serv_id: toInteger(line.hosp_serv_id)})
MATCH (d: Drug {code: line.drug_code})
MERGE (h)-[:USE]->(d)
"""
graph.run(query)

## reading data

In [2]:
neo_server = "52.15.124.208"
user = "neo4j"
passw = "digityser"

In [3]:
%config CypherMagic.uri="http://"+user+":"+passw+"@"+neo_server+":7474/db/data"

In [4]:
%load_ext cypher

In [5]:
data = %cypher \
MATCH (n:Drug) \
WHERE n.drug_name_aggregated = "IMODIUM" \
RETURN n.drug_name_aggregated, n.code_atc, n.type_drug_code

7 rows affected.


In [6]:
import pandas as pd

In [7]:
df = data.get_dataframe()
df

,n.drug_name_aggregated,n.code_atc,n.type_drug_code
0,IMODIUM,A07DA03,Package
1,IMODIUM,A07DA03,Unit
2,IMODIUM,A07DA03,Package
3,IMODIUM,A07DA03,Unit
4,IMODIUM,A07DA03,Unit
5,IMODIUM,A07DA53,Package
6,IMODIUM,A07DA03,Package
